In [ ]:
#!/usr/bin/python3
# coding: utf-8
# MHLW

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as td
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, show_and_save_plotly, \
    make_exp_fit_graph

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')

In [ ]:
# 厚労省の OpenData を参照する
base_uri = "https://www.mhlw.go.jp/content/"

raws = dict(
    posis = "pcr_positive_daily.csv",
    # 日別PCR検査人数よりも検査機関別の数値を使用すべき
    tests = "pcr_tested_daily.csv",
    cases = "cases_total.csv",
    recov = "recovery_total.csv",
    death = "death_total.csv",
    pcr = "pcr_case_daily.csv")

offsets = dict(
    dates = 0, # 日付オフセット
    cases = 1, # 入院治療を要する者(Total)
    death = 2, # 死亡者数(Total)
    pcr   = 3, # PCR検査 件数：感染研、検疫、保健所、民間、大学、医療機関(Daily), 
    pcrs  = 9, # 上記の合算
    posis = 10, # 陽性者数(Daily)
    tests = 11, # PCR検査 人数(Daily)
    recov = 12, # 退院（Total)
    ratio = 13, # 陽性率(Daily) = 陽性者数 / 検査人数
    total = 14, # 陽性者数(Total)
    ) # 

# 集計期間
dt_range = (dt.today() - dt.strptime(DT_OFFSET, "%Y/%m/%d")).days
# 配列初期化
all_data_arr = []
for i in np.arange(dt_range):
    all_data_arr.append([i, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# データの取得
for k, v in raws.items():
    download_if_needed(base_uri, v)
# データの集計
for k, v in raws.items():
    if v != 0:
        csv2array(all_data_arr, k, v, offsets[k])
# 陽性率等の計算
for i in np.arange(dt_range):
    div = all_data_arr[i][offsets['pcrs']]
    if div != 0:
        all_data_arr[i][offsets['ratio']] = max(0, min(100, (all_data_arr[i][offsets['posis']] / div) * 100))
    if i == 0:
        all_data_arr[i][offsets['total']] = all_data_arr[i][offsets['posis']]
    else:
        all_data_arr[i][offsets['total']] = all_data_arr[i][offsets['posis']] + all_data_arr[i-1][offsets['total']]

all_data_np = np.array(all_data_arr)

In [ ]:
updated = (dt.strptime(DT_OFFSET, "%Y/%m/%d") + td(days=all_data_np[-1][0])).isoformat()[:10]
with open("mhlw.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
print("updated: {}, prev: {}".format(updated, prev))
if prev == updated:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

with open("mhlw.prev.tmp", "wt") as f:
    f.write(updated)

In [ ]:
from_date = dt.strptime(DT_OFFSET, "%Y/%m/%d")
xbins = [from_date + td(days=i) for i in range(dt_range)]
days2pred = 4 * 7
xbins_pred = [from_date + td(days=i) for i in range(dt_range + days2pred)]

In [ ]:
ave_mov_days = 7
# 移動平均を算出する
posis_mov_mean = moving_average(all_data_np[:, offsets['posis']])
ratio_mov_mean = moving_average(all_data_np[:, offsets['ratio']])
print("陽性者数(移動平均): {}".format(posis_mov_mean[-1]))
print("　陽性率(移動平均): {}".format(ratio_mov_mean[-1]))

In [ ]:
X = np.arange(0, len(posis_mov_mean))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]
y_posis = get_gpr_predict(X, all_data_np[:, offsets['posis']], X_pred, 80, 10, 200)
y_ratio = get_gpr_predict(X, all_data_np[:, offsets['ratio']], X_pred, 80, 10, 200)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.xlim(xbins_pred[0], xbins_pred[-1])
    plt.ylim(0, max(all_data_np[:, offsets['posis']])+100)
    plt.plot(xbins, all_data_np[:, offsets['posis']], '.', linewidth=1, color='#c0ffc0')
    plt.plot(xbins_pred, y_posis, '-.', color='#00ff00', linewidth=1)
    plt.bar(xbins, posis_mov_mean, color='#008000', edgecolor='#008000', alpha=1)
    plt.legend(labels=["陽性者数", "予測値", "7日移動平均"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.title('全国 新型コロナ予測 陽性者数/陽性率', fontname=FONT_NAME)
    ax2 = ax.twinx()
    plt.ylim(0, 50)
    ax2.plot(xbins, all_data_np[:, offsets['ratio']], '.', linewidth=1, color='#ff00ff')
    ax2.plot(xbins_pred, y_ratio, '-.', color="#ff00ff", linewidth=1)
    ax2.bar(xbins, ratio_mov_mean, color='#f000f0', edgecolor='#f000f0', alpha=0.3)
    plt.legend(labels=["陽性率[%]", "予測値", "7日移動平均"], loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    show_and_clear(fig, "docs/images/mhlw-posis.jpg")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['posis']], mode='markers', name='陽性者数', marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_posis, mode='lines', name='予測値', line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=posis_mov_mean, name='移動平均', opacity=0.5), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['ratio']], mode='markers', name='陽性率[%]', marker=dict(size=4)), secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ratio, mode='lines', name='予測値', line=dict(width=1)), secondary_y=True)
fig.add_trace(go.Bar(x=xbins, y=ratio_mov_mean, name='移動平均', opacity=0.5), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[30], xbins_pred[-1]]),
    yaxis=dict(title='人数', range=[0, np.max(all_data_np[:, offsets['posis']])]),
    yaxis2=dict(title='陽性率[%]', range=[0,50]),
    title='全国 新型コロナ 陽性者数/陽性率 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-posis.jpg", js=False)

In [ ]:
y_tests = get_gpr_predict(X, all_data_np[:, offsets['pcrs']], X_pred, 1, 1, 5)

In [ ]:
# 移動平均を算出する
tests_mov_mean = moving_average(all_data_np[:, offsets['pcrs']])
print("検査人数(移動平均): {}".format(tests_mov_mean[-1]))
if False:
    fig, ax = create_basic_plot_figure()
    plt.xlim(xbins_pred[0], xbins_pred[-1])
    plt.plot(xbins, all_data_np[:, offsets['pcrs']], '.', linewidth=1, color='#c0ffc0')
    plt.plot(xbins_pred, y_tests, '-.', color='#00ff00', linewidth=1)
    plt.bar(xbins, tests_mov_mean, color='#008080', edgecolor='#008000', alpha=1)
    plt.legend(labels=["検査人数", "予測値", "検査人数-移動平均"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.title('全国 新型コロナ 検査人数/陽性率', fontname=FONT_NAME)
    ax2 = ax.twinx()
    plt.ylim(0, 50)
    ax2.plot(xbins, all_data_np[:, offsets['ratio']], '.', linewidth=1, color='#ff00ff')
    ax2.plot(xbins_pred, y_ratio, '-.', color="#ff00ff", linewidth=1)
    ax2.bar(xbins, ratio_mov_mean, color='#800080', edgecolor='#800080', alpha=0.5)
    plt.legend(labels=["陽性率[%]", "予測値", "7日移動平均"], loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    show_and_clear(fig, "docs/images/mhlw-tests.jpg")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['pcrs']], mode='markers', name='検査人数', marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_tests, mode='lines', name='予測値', line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=tests_mov_mean, name='移動平均', opacity=0.5), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['ratio']], mode='markers', name='陽性率[%]', marker=dict(size=4)), secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ratio, mode='lines', name='予測値', line=dict(width=1)), secondary_y=True)
fig.add_trace(go.Bar(x=xbins, y=ratio_mov_mean, name='移動平均', opacity=0.5), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[30], xbins_pred[-1]]),
    yaxis=dict(title='人数', range=[0, np.max(y_tests)]),
    yaxis2=dict(title='陽性率[%]', range=[0,50]),
    title='全国 新型コロナ 検査人数/陽性率 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-tests.jpg", js=False)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.xlim(xbins_pred[0], xbins_pred[-1])
    plt.bar(xbins, all_data_np[:, offsets['total']], color='#c08080', edgecolor='#c08080', alpha=.8)
    plt.bar(xbins, all_data_np[:, offsets['recov']], color='#00c000', edgecolor='#00c000', alpha=.7)
    plt.bar(xbins, all_data_np[:, offsets['cases']], color='#000080', edgecolor='#000080', alpha=.5)
    plt.legend(labels=["陽性者", "退院者", "入院中"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.title('全国 新型コロナ 陽性者/退院者/入院中/死者', fontname=FONT_NAME)
    ax2 = ax.twinx()
    ax2.plot(xbins, all_data_np[:, offsets['death']], color='magenta', alpha=1)
    plt.legend(labels=["死者"], loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.ylim(0, max(all_data_np[:, offsets['death']])+1)
    show_and_clear(fig, "docs/images/mhlw-total.jpg")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['total']], 
                     name='陽性者', opacity=0.8, marker_color='#c08080'), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['recov']], 
                     name='退院者', opacity=0.8, marker_color='#00c000'), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['cases']], 
                     name='入院中', opacity=0.8, marker_color='yellow'), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['death']], name="死者",
                         line=dict(width=1, color='magenta')), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[40], xbins[-1]]),
    yaxis=dict(title='人数'),
    yaxis2=dict(range=[0, np.max(all_data_np[:, offsets['death']])+10]),
    title='全国 新型コロナ 陽性者/退院者/入院中/死者 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-total.jpg", js=False)

In [ ]:
tw_body_total = "全国 新型コロナ 累計陽性者/退院者/死者(" + today_str + ") "
tw_body_total += " https://geneasyura.github.io/cov19-hm/mhlw.html "
tw_body_tests = "全国 新型コロナ 検査人数/陽性率(" + today_str + ") "
tw_body_tests += " https://geneasyura.github.io/cov19-hm/mhlw.html "
tw_body_posis = "全国 新型コロナ 陽性者/陽性率(" + today_str + ") "
tw_body_posis += " https://geneasyura.github.io/cov19-hm/mhlw.html "

In [ ]:
tw = get_twitter()
tweet_with_image(tw, "docs/images/mhlw-posis.jpg", tw_body_posis)
tweet_with_image(tw, "docs/images/mhlw-tests.jpg", tw_body_tests)
tweet_with_image(tw, "docs/images/mhlw-total.jpg", tw_body_total)

In [ ]:
# 実効再生産数
ogiwara_uri = "https://raw.githubusercontent.com/kaz-ogiwara/covid19/master/data/"
ern_file = "effective_reproduction_number.csv"
download_if_needed(ogiwara_uri, ern_file)

In [ ]:
ern_data_arr = []
for i in np.arange(dt_range):
    ern_data_arr.append([i, 0, 0, 0])
csv2array(ern_data_arr, 'ern', ern_file, 1)
ern_data_np = np.array(ern_data_arr)
#print(ern_data_np[:,1])

In [ ]:
y_ern = get_gpr_predict(X, ern_data_np[:, 1], X_pred, 80, 10, 200)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.xlim(xbins_pred[44], xbins_pred[-1])
    plt.ylim(0, max(ern_data_np[:, 1])+0.1)
    plt.ylabel('実効再生産数', fontname=FONT_NAME)
    plt.plot(xbins_pred, y_ern, '-.', linewidth=1, color='#ff8080')
    plt.bar(xbins, ern_data_np[:, 1], color='#00c000', edgecolor='#00c000', alpha=.5)
    plt.legend(labels=["予測値", "実効再生産数"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.title('全国 新型コロナ予測 実効再生産数', fontname=FONT_NAME)
    show_and_clear(fig, "docs/images/ogiwara-ern.jpg")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=xbins, y=ern_data_np[:, 1], name="実効再生産数", opacity=0.5))
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ern, mode='lines', name='予測値', line=dict(width=1)))
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins[44], xbins_pred[-1]]),
    yaxis=dict(title='実効再生産'),
    title='全国 新型コロナ 実効再生産数 ({})'.format(today_str),
)
show_and_save_plotly(fig, "ogiwara-ern.jpg", js=False)

In [ ]:
tw_body_ern = "全国 新型コロナ 実効再生産数 ({})".format(today_str)
tw_body_ern += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tweet_with_image(tw, "docs/images/ogiwara-ern.jpg", tw_body_ern)

In [ ]:
title = '全国 新型コロナ 新規陽性者移動平均/累乗近似 (' + today_str + ')'
xdata = np.array(xbins)
#ydata = all_data_np[:, offsets['posis']]
ydata = posis_mov_mean
xos = 265
make_exp_fit_graph(tw,
    xdata[xos:], ydata[xos:],
    title, "mhlw-fit.jpg",
    "mhlw-doubling-time.html", "mhlw.html")